In [1]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
import boto3
import sys
import re
from datetime import datetime
import configparser

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1668401719806_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%configure -f
{
    "conf": {
        "spark.jars": "s3://sjet-datamart-bucket/Jars/mssql-jdbc-6.1.0.jre8.jar",
        "spark.sql.legacy.parquet.int96RebaseModeInWrite": "CORRECTED"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1668141925435_0008,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1668141925435_0006,pyspark,idle,Link,Link,
6,application_1668141925435_0008,pyspark,idle,Link,Link,✔


In [3]:
GoldenRecordISPINC_fact = spark.sql('''
                                        SELECT * FROM 
                                        22_customerhub.goldenrecordispinc_fact
''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
GoldenRecordISPINC_fact.createOrReplaceTempView('GoldenRecordISPINC_fact')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
tmpBookingdate = spark.sql('''
                            SELECT DISTINCT cast(max(bookingdate) AS DATE) AS MAXBookingDate

                            ,org_uid

                            FROM GoldenRecordISPINC_fact

                            GROUP BY org_uid 
                            
                            ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
tmpBookingdate.createOrReplaceTempView('tmpBookingdate')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
tmpDepartureDate = spark.sql('''

                SELECT DISTINCT cast(max(DepartureDate_PS) AS DATE) AS MAXDepartureDate, org_uid

                FROM GoldenRecordISPINC_fact 

                GROUP BY org_uid
                
                ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
tmpDepartureDate.createOrReplaceTempView('tmpDepartureDate_PS')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#SELECT cast(dateadd(month, datediff(month, 0, MAXBookingDate) - 12, 0) AS DATE) FROM ##tmpBookingdate where org_uid = '16116833142100000003'

spark.sql('''
                SELECT add_months('1900-01-01', CAST(months_between(tdd.MAXDepartureDate, '1900-01-01') AS INT) -12) AS Date
                            
                            FROM GoldenRecordISPINC_fact g
                            
                            left join tmpDepartureDate_PS tdd on tdd.org_uid = g.org_uid
            ''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|      Date|
+----------+
|2014-10-01|
|2014-10-01|
|2014-11-01|
|2014-11-01|
|2014-11-01|
|2014-11-01|
|2014-11-01|
|      null|
|2014-11-01|
|2014-11-01|
|2014-11-01|
|2014-12-01|
|2014-12-01|
|      null|
|2014-11-01|
|2014-11-01|
|2015-01-01|
|2015-01-01|
|2015-01-01|
|2015-01-01|
+----------+
only showing top 20 rows

In [89]:
df1 = spark.sql('''
                    SELECT max(DepartureDate_PS) AS MaxDepartureDate_PS FROM GoldenRecordISPINC_fact
''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
df1.createOrReplaceTempView('c')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
#SELECT dateadd(month, datediff(month, 0, (SELECT max(DepartureDate_PS) FROM CUSTOMERHUB.DBO.GoldenRecordISPINC_fact)) - 12, 0)

spark.sql('''
                SELECT add_months('1900-01-01', 
                CAST(months_between(
                                (SELECT max(DepartureDate_PS) 
                                FROM GoldenRecordISPINC_fact t2 WHERE t2.ORG_UID = g.org_uid), '1900-01-01') AS INT) -12)  
                
                FROM GoldenRecordISPINC_fact g

''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|MaxBookingDate|
+--------------+
|    2017-05-01|
|    2017-05-01|
|    2017-05-01|
+--------------+